# CISC3024 Pattern Recognition Final Project
## Group Members:
- Huang Yanzhen, DC126732
- Mai Jiajun, DC12785

## 0. Project Setup

### 0.1 Packages & Device

In [37]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

import numpy as np
import cv2
import os

In [38]:
device_name = "cuda" if torch.cuda.is_available() else "cpu"
device = torch.device(device_name)
print(f"Using device: {device_name}")

Using device: cuda


### 0.2 Global Configurations

In [39]:
path_dataset = "./data/SVHN_mat"

## 1. Data Processing and Augmentation

### 1.1 Download Datasets

In [32]:
import albumentations as A
from albumentations.pytorch import ToTensorV2
import scipy.io as sio

In [33]:
transform = A.Compose([
    A.RandomResizedCrop(32, 32),
    A.Rotate(limit=30),
    A.Normalize(mean=[0.4377, 0.4438, 0.4728], std=[0.1980, 0.2010, 0.1970]),
    ToTensorV2()
])

In [43]:
class SVHNDataset(Dataset):
    def __init__(self, mat_file, transform=None):
        data = sio.loadmat(mat_file)
        self.images = np.transpose(data['X'], (3, 0, 1, 2))
        self.labels = data['y'].flatten()
        self.labels[self.labels == 10] = 0
        self.transform = transform

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]

        if self.transform:
            # image = self.transform(image)
            image = self.transform(image=image)['image']

        return image, label

In [40]:
train_dataset = SVHNDataset(mat_file=os.path.join(path_dataset,"train_32x32.mat"), transform=transform)
test_dataset = SVHNDataset(mat_file=os.path.join(path_dataset,"test_32x32.mat"), transform=transform)
extra_dataset = SVHNDataset(mat_file=os.path.join(path_dataset,"extra_32x32.mat"), transform=transform)

In [44]:
train_dataset.__getitem__(0)

KeyError: 'You have to pass data to augmentations as named arguments, for example: aug(image=image)'